In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

In [ ]:
import os
llm = Groq(model="llama-3.3-70b-versatile",
           api_key=os.environ.get("GROQ_API_KEY"))

In [ ]:
def calcular_imposto_renda(rendimento: float) -> str:
    """ 
    Calcular o imposto de renda com base no rendimento anual.

    Args:
        rendimento (float): Rendimento anual do indivíduo.

    Returns:

        str: O valor do imposto devido com base no rendimento
    
    """
    if rendimento <= 2000:
        return " Voce está isento de pagar imposto de renda"
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000 ) * 0.1
        return f"O valor do imposto devido é R${imposto:.2f}, base em no rendimento de R$ {rendimento:.2f}"
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"O valor do imposto devido é R${imposto:.2f}, base em no rendimento de R$ {rendimento:.2f}" 
    else:
        imposto = (rendimento - 10000) * 0.20 + 1500
        return f"O valor do imposto devido é R${imposto:.2f}, base em no rendimento de R$ {rendimento:.2f}"

## Convertendo a função em Ferramenta

In [ ]:
from llama_index.core.tools import FunctionTool

In [ ]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name= "Calcular Imposto de Renda",
    description=(
        "Calcula o imposto de renda de acordo com o rendimento atual."
        "Argumento: rendimento (float)."
        "Retorna o valor do imposto devido de acordo com as faixas de rendimento"
    )
)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [ ]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [ ]:
from llama_index.core.agent import AgentRunner

In [ ]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [ ]:
response = agent_imposto.chat("""
    Qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7.500?
]
                              """)

In [ ]:
# Testando que a LLM não irá passar pela tool criada acima
response = agent_imposto.chat("Quem foi Machado de Assis?")

In [ ]:
import arxiv

def consulta_artigos(titulo: str) -> str:
    """Consulta os artigos na base de dados ArXiv e retorna resultados formatados."""
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]

    return "\n\n".join(resultados)

In [ ]:
# Convertendo a função para tool
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [ ]:
agent = AgentRunner(agent_worker)
agent.chat("Me retorne artigos sobre LangChain na educação ")

## Usando Tavily

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
   
dotenv = load_dotenv(find_dotenv())

In [ ]:
tavily_key = os.environ.get("TAVILY_API_KEY")

In [ ]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

In [ ]:
# Função de busca
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

In [ ]:
tavily_tool.search("Me retorne artigos sobre langchain", max_results=3)

## Transformando API de terceiro como ferramenta para o agente

In [ ]:
from llama_index.core.tools import FunctionTool

tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily Search",
    description="Busca artigos Tavily sobre um determinado topico"
)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [ ]:
agent = AgentRunner(agent_worker)
response = agent.chat("Me retorne artigos sobre LLM e LangChain")

In [ ]:
print(response)

## Llama com leitura de arquivo (uso de RAG)

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [ ]:
url = "../files/LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

In [ ]:
url = "../files/LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

## Gerar os Embeddings

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name = "intfloat/multilingual-e5-large"
)

In [ ]:
artigo_index = VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

In [ ]:
# Cria um diretorio para o artigo e tutorial
artigo_index.storage_context.persist(persist_dir="artigo")
tutorial_index.storage_context.persist(persist_dir="tutorial")

## Engine de Busca

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

In [ ]:
storage_context = StorageContext.from_defaults(
    persist_dir="artigo"
)

artigo_index = load_index_from_storage(storage_context)

In [ ]:
storage_context = StorageContext.from_defaults(
    persist_dir="tutorial"
)

tutorial_index = load_index_from_storage(storage_context)

In [ ]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3, llm=llm)
tutorial_engine = tutorial_index.as_query_engine(similarity_top_k=3, llm=llm)

## Testando o Agent

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine, 
        metadata=ToolMetadata(
            name="artigo_engine",
            description="Fornece informações sobre LLM e LangChain."
            "Use uma pergunta simples e detalhada em texto simples como entrada para a ferramenta"
        )
    ),
        QueryEngineTool(
        query_engine=tutorial_engine, 
        metadata=ToolMetadata(
            name="tutorial_engine",
            description="Fornece informações sobre casos de uso e aplicações em LLMs."
            "Use uma pergunta simples e detalhada em texto simples como entrada para a ferramenta"
        )
    )
]

In [ ]:
# No caso abaixo faz sentido o uso de allow_parallel_tool_calls = True pois o agente irá consultar os 2 pdfs ao mesmo tempo
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

In [ ]:
# Abaixo será possivel verificar que o agente percorrer os 2 documentos para responder a pergunta.
response = agent_document.chat("Quais as principais aplicações que posso contruir com LLM e LangChain")

## Agente React
Usando agentes reativos para testes. 
 - Possui um tempo de respostas mais rápido.

In [ ]:
from llama_index.core.agent import ReActAgent

In [ ]:
agente = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    llm=llm
)

In [ ]:
response = agente.chat("Quais as principais ferramentas usadas em LangChain?")